# Segmenting and Clustering of the Neighborhoods of Toronto

The data are on a Wikipedia table, at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Web scraping --> I will use the Beautiful Soup library to get the data from the table

In [191]:
# import the request library
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(url).text

# Prettify() function in BeautifulSoup will enable us to view how the tags are nested in the document
soup = BeautifulSoup(website_url,'lxml')
# print just few characters to see how it looks like
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

### Extract the table using the soup.find. Then, extract all the <td> ... </td> which contain the postcodes, boroughs and neighborhoods

In [221]:
# extract the table
mytable = soup.find('table',{'class':'wikitable sortable'})
# extract the rows that start with <td>
tdALL   = mytable.find_all('td')
# print the first 5 just to check
print(tdALL[:5])

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
</td>, <td>M2A</td>, <td>Not assigned</td>]


### Loop in the tdALL and extract the data where Borough is NOT 'Not Assigned' 

In [223]:
postcode = []
borough  = []
neighborhood = []
for ii in range(0,len(tdALL)-3,3):
    if "Not" not in tdALL[ii+1].text:
        postcode.append(tdALL[ii].text)
        borough.append(tdALL[ii+1].text)
        neighborhood.append(tdALL[ii+2].text)        

### Check what the 3 lists look like

In [226]:
print(postcode[:5])
print(borough[:5])
print(neighborhood[:5])

['M3A', 'M4A', 'M5A', 'M5A', 'M6A']
['North York', 'North York', 'Downtown Toronto', 'Downtown Toronto', 'North York']
['Parkwoods\n', 'Victoria Village\n', 'Harbourfront\n', 'Regent Park\n', 'Lawrence Heights\n']


### Use the lists just found to create a dataframe

In [227]:
# create a dataframe with PostalCode, Borough, and Neighborhood using the lists found above
import pandas as pd
df = pd.DataFrame()
df['PostalCode']   = postcode
df['Borough']      = borough
df['Neighborhood'] = neighborhood

# strip off the '\n' from the Neighborhood column
df['Neighborhood'] = df['Neighborhood'].map(lambda x: x.rstrip('\n'))
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Shape of the dataframe

In [228]:
df.shape

(212, 3)

### Create a new dataframe, grouping by PostalCode

In [230]:
# group by PostalCode
# get the arrays with the values for Boroughs and Neighborhoods, given a unique PostalCode
df2    = df[['PostalCode','Borough','Neighborhood']].groupby('PostalCode')
l1     = df2.apply(lambda x: x['Neighborhood'].unique())
l2     = df2.apply(lambda x: x['Borough'].unique())                                            
# create a dictionary with the 2 lists
d      = {'Borough':l2,'Neighborhood':l1}
dfnew  = pd.DataFrame(d)
## or:
##dfnew = pd.DataFrame({k: v for k, v in d.items()})
dfnew.reset_index(level=0, inplace=True)
dfnew.rename(columns={'index': 'PostalCode'},inplace=True)
dfnew['Borough'] = dfnew['Borough'].str.get(0)
dfnew.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


### Shape of the new dataframe:

In [231]:
dfnew.shape

(103, 3)

In [196]:
# check 1 row (M5A), to see that it gives the same values as in the directions of the assignement
dfnew.loc[dfnew['PostalCode']=='M4V']

,PostalCode,Borough,Neighborhood
49,M4V,Central Toronto,"[Deer Park, Forest Hill SE, Rathnelly, South H..."


In [239]:
#OR (from Nobuaki Ito):
df3        = pd.read_html(str(mytable))[0]
new_header = df3.iloc[0] 
df3        = df3[1:] 
df3.columns= new_header
df3        = df3.replace('Not assigned',np.NaN)
df3        = df3.dropna(subset=['Borough'])
df3        = df3.reset_index(drop=True)
for i, row in my_df.iterrows():
    if pd.isnull(df3.at[i,'Neighbourhood']):
        df3.at[i,'Neighbourhood'] = df3.at[i,'Borough']
        
df3 = df3.groupby(['Postcode','Borough'],as_index=False).agg({'Neighbourhood':', '.join})
print(df3.shape)
df3.head()

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
